In [1]:
import pandas as pd 
import os
import plotly.express as px # version ()
import plotly.graph_objects as go

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output


# importing and clean data ( importing csv into pandas)

In [13]:

# Set your path to the folder containing the .csv files
PATH = '../data/Analysis/' # Use your path

# Fetch all files in path
fileNames = os.listdir(PATH)

# Filter file name list for files ending with .csv
fileNames = [file for file in fileNames if '_results.csv' in file]

# Loop over all files

my_list = []

for file in fileNames:
# Read .csv file and append to list
    df = pd.read_csv(PATH + file, index_col= None, header=0)
    df['Name'] = file.replace('_results.csv','')
    my_list.append(df)  


In [ ]:
energy_con_dataframe = pd.concat(my_list, axis=0, ignore_index=True)
energy_con_dataframe = energy_con_dataframe.drop(columns=['obs_ci_lower','obs_ci_upper']).round(1)
energy_con_dataframe.reset_index(inplace=True)

In [15]:
energy_con_dataframe['Datetime'] = pd.to_datetime(energy_con_dataframe['Datetime'], utc=True, errors='ignore')


In [18]:
energy_con_dataframe['year'] = energy_con_dataframe.Datetime.dt.year
energy_con_dataframe['month'] = energy_con_dataframe.Datetime.dt.month
energy_con_dataframe['days'] = energy_con_dataframe.Datetime.dt.day

## Cleaning up data and creating lables for graphing

In [ ]:
Name = energy_con_dataframe['Name']
yearly = energy_con_dataframe('year').unique()
mothly = energy_con_dataframe['month']
daily = energy_con_dataframe['days']
actual = energy_con_dataframe['Actual']
predict = energy_con_dataframe['Predicted']


# App Layout

In [31]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1(children = 'UNCG Engergy Consumptions',
           style={
               'textAlign' : 'center',
               'color' : '#456FBV'
           }),
    html.Div(children='funded by Green Fund',
            style={
                'textAlign' : 'center',
                'color' : '#456FBV'
            }),
    
    dcc.Graph(
        id='overall-consumption-graph',
        figure={
            'data': [
                {'x': yearly, 'y':actual.mean(), 'type': 'bar', 'name': 'Actual'},
                {'x': yearly, 'y': predict.mean(), 'type': 'bar', 'name': u'Predicted'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

<!-- # mgr_options = energy_con_dataframe["Name"].unique()

# app = dash.Dash()

# app.layout = html.Div([
#     html.H2("Sales Funnel Report"),
#     html.Div(
#         [
#             dcc.Dropdown(
#                 id="Name",
#                 options=[{
#                     'label': i,
#                     'value': i
#                 } for i in mgr_options],
#                 value='All Managers'),
#         ],
#         style={'width': '25%',
#                'display': 'inline-block'}),
#     dcc.Graph(id='funnel-graph'),
# ])


# @app.callback(
#     dash.dependencies.Output('funnel-graph', 'figure'),
#     [dash.dependencies.Input('Name', 'value')])
# def update_graph(Manager):
#     if Manager == "All Managers":
#         df_plot = df.copy()
#     else:
#         df_plot = df[df['Name'] == Manager]

#     pv = pd.pivot_table(
#         df_plot,
#         index=['Name'],
#         columns=["Datetime"],
#         values=['Actual'],
#         aggfunc=sum,
#         fill_value=0)

#     trace1 = go.Bar(x=pv.index, y=pv[('Quantity', 'declined')], name='Declined')
#     trace2 = go.Bar(x=pv.index, y=pv[('Quantity', 'pending')], name='Pending')
#     trace3 = go.Bar(x=pv.index, y=pv[('Quantity', 'presented')], name='Presented')
#     trace4 = go.Bar(x=pv.index, y=pv[('Quantity', 'won')], name='Won')

#     return {
#         'data': [trace1, trace2, trace3, trace4],
#         'layout':
#         go.Layout(
#             title='Customer Order Status for {}'.format(Manager),
#             barmode='stack')
#     }
 -->

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Nov/2020 02:48:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2020 02:48:48] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2020 02:48:56] "GET /_favicon.ico?v=1.17.0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2020 02:49:10] "GET /_dash-layout HTTP/1.1" 200 -
